In [3]:
import numpy as np
import numpy.random 
from collections import defaultdict
import xgboost as xgb
import gtnlplib.preproc
import gtnlplib.constants
from nltk.tokenize import word_tokenize, sent_tokenize
import os
import shutil

In [6]:
if os.path.exists('data/train_sentences'):
    shutil.rmtree('data/train_sentences')
os.mkdir('data/train_sentences')

In [7]:
fs_key = open('data/train_sentences.key', 'w')
with open(gtnlplib.constants.TRAINKEY, 'r') as f_idxes:
    root_folder = gtnlplib.constants.TRAINKEY.split('/')[0]
    for name_label in f_idxes:
        filename, label = name_label.split(' ')
        file_prefix = filename.split('/')[1].split('.')[0]
        filename = root_folder + '/' + filename
        label = label.strip()
        with open(filename.strip(), 'r') as f:
            for line in f:
                decoded = line.decode('ascii','ignore')
                sentences = sent_tokenize(decoded)
                s_idx = 0
                for s in sentences:
                    words = word_tokenize(s)
                    cnt = 0
                    for w in words:
                        if w.isalpha():
                            cnt += 1
                    if cnt == 0:
                        continue                    
                    subfile = 'train_sentences/%s_%d.txt' % (file_prefix, s_idx)
                    with open('data/' + subfile, 'w') as subf:
                        subf.write(s)
                    fs_key.write(subfile + ' ' + label + '\n')
                    s_idx += 1
fs_key.close()


In [10]:
if os.path.exists('data/train_crop'):
    shutil.rmtree('data/train_crop')
os.mkdir('data/train_crop')

times = 20
fraction = 0.5
fs_key = open('data/train_crop.key', 'w')
with open(gtnlplib.constants.TRAINKEY, 'r') as f_idxes:
    root_folder = gtnlplib.constants.TRAINKEY.split('/')[0]
    for name_label in f_idxes:
        filename, label = name_label.split(' ')
        file_prefix = filename.split('/')[1].split('.')[0]
        filename = root_folder + '/' + filename
        label = label.strip()
        with open(filename.strip(), 'r') as f:
            for line in f:
                decoded = line.decode('ascii','ignore')
                sentences = sent_tokenize(decoded)
                wlist = []
                for s in sentences:
                    words = word_tokenize(s)                    
                    for w in words:
                        if w.isalpha():
                            wlist.append(w)
                            
                for s_idx in range(times):
                    subfile = 'train_crop/%s_%d.txt' % (file_prefix, s_idx)
                    with open('data/' + subfile, 'w') as subf:
                        for i in range(len(wlist)):
                            if len(wlist) < 10 or np.random.rand() > fraction:
                                subf.write(wlist[i] + ' ')
                        subf.write('\n')
                    fs_key.write(subfile + ' ' + label + '\n')
fs_key.close()

In [7]:
if os.path.exists('data/train_randconcat'):
    shutil.rmtree('data/train_randconcat')
os.mkdir('data/train_randconcat')

times = 1000
fs_key = open('data/train_randconcat.key', 'w')
sample_dict = {'POS' : [], 'NEG' : [], 'NEU' : []}

with open(gtnlplib.constants.TRAINKEY, 'r') as f_idxes:
    root_folder = gtnlplib.constants.TRAINKEY.split('/')[0]
    for name_label in f_idxes:
        filename, label = name_label.split(' ')
        file_prefix = filename.split('/')[1].split('.')[0]
        filename = root_folder + '/' + filename
        label = label.strip()
        with open(filename.strip(), 'r') as f:
            for line in f:
                decoded = line.decode('ascii','ignore')
                sentences = sent_tokenize(decoded)
                sample_dict[label].append(sentences)
                wlist = []
                for s in sentences:
                    words = word_tokenize(s)
                    for w in words:
                        if w.isalpha():
                            wlist.append(w)
                subfile = 'train_randconcat/%s.txt' % file_prefix
                with open('data/' + subfile, 'w') as subf:
                    for i in range(len(wlist)):
                        subf.write(wlist[i] + ' ')
                    subf.write('\n')
                fs_key.write(subfile + ' ' + label + '\n')
                
for label in sample_dict:
    for t in range(times):
        subfile = 'train_randconcat/%s_%d.txt' % (label, t)
        idx1 = np.random.randint(len(sample_dict[label]))
        idx2 = np.random.randint(len(sample_dict[label]))
        if idx1 == idx2:
            continue
        s1 = sample_dict[label][idx1]
        s2 = sample_dict[label][idx2]
        fs_key.write(subfile + ' ' + label + '\n')
        with open('data/' + subfile, 'w') as subf:
            wlist = []
            for i in range(len(s1) / 2 + 1):
                subf.write(s1[i] + ' ')
            for i in range(len(s2) / 2 + 1):
                subf.write(s2[i] + ' ')
            subf.write(' '.join(wlist) + '\n')
fs_key.close()